In [0]:
CREATE OR REFRESH STREAMING TABLE bronze_orders
TBLPROPERTIES('quality'='bronze')
COMMENT 'orders bronze table'
AS
select *
      ,_metadata.file_path as file_path
      ,current_timestamp as injested_timestamp
FROM cloud_files(
      '/Volumes/dlt_pipeline_catalog/landing_schema/project_dlt_volume/orders/',
      'json',
      map(
        'cloudFiles.inferColumnTypes','true',
        'cloudFiles.schemaEvolutionMode','addNewColumns'  
      )
      
      
      )

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_orders_clean
(
  CONSTRAINT valid_customer_id EXPECT(customer_id IS not null) on VIOLATION FAIL UPDATE,
  CONSTRAINT valid_order_id EXPECT(order_id IS not null) on VIOLATION FAIL UPDATE,
  constraint check_payment_status expect(order_status in ('Pending','Shipped','Cancelled','Completed')),
constraint check_payment_method expect(payment_method in ('Credit Card','Bank Transfer', 'PayPal'))
)
comment 'cleaned orders table'
tblproperties('quality' = 'silver')
as select order_id,
          customer_id,
          cast(order_timestamp as date) as order_timestamp,
          payment_method,
          explode(items) AS item,
          order_status
from stream(LIVE.bronze_orders);




In [0]:
CREATE OR REFRESH STREAMING table silver_orders
COMMENT 'silver table for orders'
TBLPROPERTIES('quality'='silver')
AS
SELECT order_id
      customer_id,
      cast(order_timestamp as date) as order_timestamp,
      payment_method,
      order_Status,
      item.item_id,
      item.name,
      item.category,
      item.price,
      item.quantity
FROM STREAM(LIVE.SILVER_ORDERS_CLEAN)